# Comparação do Vader com outros algoritmos para análise do sentimento

- Autor: Daniel Saraiva Leite - 2023
- Projeto Análise de sentimentos sobre notícias do tema ESG
- Trabalho de conclusão de curso - MBA Digital Business USP Esalq

In [11]:
from analise_sentimento_modelo import pondera_polaridade_titulo_texto, classifica_sentimento_vaderptbr, polaridade_sentimento_vaderptbr
from vaderSentimentptbr import SentimentIntensityAnalyzer 
from sumarizador_textrankptbr import summarize_text_rank 
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd
import spacy
import nltk
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn import svm
import numpy as np
from unidecode import unidecode
from noticias_processamento_texto import lematizador
     

# Funçoes de avaliação de cada algoritmo

In [12]:
'''
Realiza a aplicação de cada algortimo no dataset, indicando suas métricas de Acurácia, Precisão, Cobertura e F1-measure
Para algortimos de machine learing, aplica 5-fold cross-validation
'''
def avalia_algoritmo(df, col_texto, col_titulo=None, col_sentimento='sentimento', sent_pos=1, sent_neu=0, sent_neg=-1, modelo='Vader', resumir=True):
    
    # vader - nao supervisionado
    
    if modelo == 'Vader':
        if col_titulo is not None:
            df['sentimento_vader'] = df.apply(lambda row: classifica_sentimento_vaderptbr(polaridade_sentimento_vaderptbr(row[col_texto], row[col_titulo], resumir), sent_pos, sent_neu, sent_neg) , axis=1)
        else:
            df['sentimento_vader'] = df.apply(lambda row: classifica_sentimento_vaderptbr(polaridade_sentimento_vaderptbr(row[col_texto], None, resumir), sent_pos, sent_neu, sent_neg) , axis=1)
        print('Accuracy Score: %.3f' % accuracy_score(df[col_sentimento], df['sentimento_vader']))
        print('Precision Score: %.3f' % precision_score(df[col_sentimento], df['sentimento_vader'], average='binary'))
        print('Recall Score: %.3f' % recall_score(df[col_sentimento], df['sentimento_vader'], average='binary'))
        print('F1 Score: %.3f' % f1_score(df[col_sentimento], df['sentimento_vader'], average='binary'))
        return 
    
    # machine learning com cross-validation
    
    df['texto_ajustado'] = df[col_texto].apply(str.lower)
    df['texto_ajustado'] = df['texto_ajustado'].apply(lematizador)

    X, y = df['texto_ajustado'], df[col_sentimento]
    metrics_acc = []
    metrics_p = []
    metrics_r = []
    metrics_f1 = []

    skf = StratifiedKFold(n_splits=5, shuffle=True)
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        vect = TfidfVectorizer(ngram_range=(1,2), max_features=1000 , stop_words=list(spacy.lang.pt.stop_words.STOP_WORDS) + ['abc', 'xyz', 'def'])
        X_train_dtm = vect.fit_transform(X_train)
        X_test_dtm = vect.transform(X_test)
        model=None
        if modelo =='MultinomialNB':
            model = MultinomialNB()
        elif modelo == 'LogisticRegression':
            model = LogisticRegression(max_iter=200)
        else:
            model = svm.SVC(kernel='linear')
        model.fit(X_train_dtm, y_train)
        y_pred_class = model.predict(X_test_dtm)

        metrics_acc.append(accuracy_score(y_test, y_pred_class))
        metrics_p.append(precision_score(y_test, y_pred_class, average='binary'))
        metrics_r.append(recall_score(y_test, y_pred_class, average='binary'))
        metrics_f1.append(f1_score(y_test, y_pred_class, average='binary'))

    metrics_acc = np.array(metrics_acc)
    metrics_p = np.array(metrics_p)
    metrics_r = np.array(metrics_r)
    metrics_f1 = np.array(metrics_f1)
    print('Media accuracy: ', np.mean(metrics_acc, axis=0))
    print('Media precision: ', np.mean(metrics_p, axis=0))
    print('Media recall: ', np.mean(metrics_r, axis=0))
    print('Media f1: ', np.mean(metrics_f1, axis=0))
    
    

# Comparação com dataset do trabalho de Magner et al. (2022)

In [13]:
print('Comparação com trabalho: Magner, Renata; Medeiros, Camila Alves; Felix, Klysman; Souza, Pedro Vieira. 2020. Análise de Sentimentos de Informações do Mercado Financeiro — Projeto de Conclusão de Curso — Data Science & Machine Learning da Tera. Disponível em: <https://medium.com/@renata.amp/análise-de-sentimentos-de-informações-do-mercado-financeiro-projeto-de-conclusão-de-curso-data-6a8f8dcbf441>\n')
df = pd.read_excel('datasets/Dataset_noticias_tera.xlsx')

print('\nAvaliando base Magner et al com VADER')
avalia_algoritmo(df, 'texto_noticia', col_titulo='titulo', col_sentimento='sentimento', sent_pos=1, sent_neu=1, sent_neg=0, modelo='Vader')

print('\nAvaliando base Magner et al com MultinomialNB')
avalia_algoritmo(df, 'texto_noticia', col_titulo='titulo', col_sentimento='sentimento', sent_pos=1, sent_neu=1, sent_neg=0, modelo='MultinomialNB')
    
print('\nAvaliando base Magner et al com LogisticRegression')
avalia_algoritmo(df, 'texto_noticia', col_titulo='titulo', col_sentimento='sentimento', sent_pos=1, sent_neu=1, sent_neg=0, modelo='LogisticRegression')

print('\nAvaliando base Magner et al com SVM')
avalia_algoritmo(df, 'texto_noticia', col_titulo='titulo', col_sentimento='sentimento', sent_pos=1, sent_neu=1, sent_neg=0, modelo='SVC')

Comparação com trabalho: Magner, Renata; Medeiros, Camila Alves; Felix, Klysman; Souza, Pedro Vieira. 2020. Análise de Sentimentos de Informações do Mercado Financeiro — Projeto de Conclusão de Curso — Data Science & Machine Learning da Tera. Disponível em: <https://medium.com/@renata.amp/análise-de-sentimentos-de-informações-do-mercado-financeiro-projeto-de-conclusão-de-curso-data-6a8f8dcbf441>


Avaliando base Magner et al com VADER


/Users/danielsaraivaleite/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Accuracy Score: 0.708
Precision Score: 0.740
Recall Score: 0.856
F1 Score: 0.794

Avaliando base Magner et al com MultinomialNB
Media accuracy:  0.6937010954616588
Media precision:  0.7091720921700094
Media recall:  0.9068262411347519
Media f1:  0.7955468303106132

Avaliando base Magner et al com LogisticRegression
Media accuracy:  0.6991784037558686
Media precision:  0.6983867540211794
Media recall:  0.9574468085106382
Media f1:  0.8070111870111869

Avaliando base Magner et al com SVM
Media accuracy:  0.7213615023474179
Media precision:  0.7315693233730819
Media recall:  0.9154255319148936
Media f1:  0.8123318038261989


# Comparação com dataset sintético gerado pela API do Chat GPT

In [14]:
print('\n\nAvaliando com base noticias geradas pela API do ChatGPT')
df = pd.read_excel('datasets/gpt_noticias_ESG.xlsx')
df = df[df.Sentimento != 'Neutra']
df['Sentimento'] = df.Sentimento.apply(lambda x : 1 if x == 'Positiva' else -1)
df = df.reset_index(drop=True)

print('\nAvaliando noticias do ChatGPT com VADER')
avalia_algoritmo(df, 'Texto', col_titulo=None, col_sentimento='Sentimento', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='Vader')

print('\nAvaliando noticias do ChatGPT com MultinomialNB')
avalia_algoritmo(df, 'Texto', col_titulo=None, col_sentimento='Sentimento', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='MultinomialNB')
    
print('\nAvaliando noticias do ChatGPT com LogisticRegression')
avalia_algoritmo(df, 'Texto', col_titulo=None, col_sentimento='Sentimento', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='LogisticRegression')
    
print('\nAvaliando noticias do ChatGPT com SVM')
avalia_algoritmo(df, 'Texto', col_titulo=None, col_sentimento='Sentimento', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='SVC')
    



Avaliando com base noticias geradas pela API do ChatGPT

Avaliando noticias do ChatGPT com VADER
Accuracy Score: 0.929
Precision Score: 0.948
Recall Score: 0.937
F1 Score: 0.942

Avaliando noticias do ChatGPT com MultinomialNB
Media accuracy:  0.9666666666666668
Media precision:  0.959245418112695
Media recall:  0.988956846755012
Media f1:  0.973695819457765

Avaliando noticias do ChatGPT com LogisticRegression
Media accuracy:  0.9724137931034484
Media precision:  0.9593178397987941
Media recall:  0.998165137614679
Media f1:  0.9783284704819757

Avaliando noticias do ChatGPT com SVM
Media accuracy:  0.9804597701149426
Media precision:  0.9730990636105519
Media recall:  0.9963132857628271
Media f1:  0.9845240979437284


# Comparação no dataset traduzido de notícias financeiras FinancialPhraseBank
Malo, P., Sinha, A., Korhonen, P., Wallenius, J., & Takala, P. (2014). Good debt or bad debt: Detecting semantic orientations in economic texts. Journal of the Association for Information Science and Technology, 65(4), 782-796.
https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news

In [15]:
import deep_translator
import string
from deep_translator import GoogleTranslator
# traducao do dataset - descomentar esse trecho para executar
#gt = GoogleTranslator(source='en', target='pt')
#df = pd.read_csv('FinancialPhraseBank.csv', encoding='ISO-8859-1', header=None, names=['sentiment', 'text'])
#df['translation'] = df['text'].apply(gt.translate)
#df.to_excel('FinancialPhraseBank_translated.xlsx', index=False)
df = pd.read_excel('datasets/FinancialPhraseBank_translated.xlsx')
df = df[df.sentiment != 'neutral']
df['sentiment'] = df.sentiment.apply(lambda x : 1 if x == 'positive' else -1)
df = df.reset_index()

print('\nAvaliando FinancialPhraseBank com VADER')
avalia_algoritmo(df, 'translation', col_titulo=None, col_sentimento='sentiment', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='Vader', resumir=False)

print('\nAvaliando FinancialPhraseBank com MultinomialNB')
avalia_algoritmo(df, 'translation', col_titulo=None, col_sentimento='sentiment', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='MultinomialNB', resumir=False)
    
print('\nAvaliando FinancialPhraseBank LogisticRegression')
avalia_algoritmo(df, 'translation', col_titulo=None, col_sentimento='sentiment', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='LogisticRegression', resumir=False)
    
print('\nAvaliando FinancialPhraseBank SVM')
avalia_algoritmo(df, 'translation', col_titulo=None, col_sentimento='sentiment', sent_pos=1, sent_neu=1, sent_neg=-1, modelo='SVC', resumir=False)
    



Avaliando FinancialPhraseBank com VADER
Accuracy Score: 0.743
Precision Score: 0.782
Recall Score: 0.872
F1 Score: 0.824

Avaliando FinancialPhraseBank com MultinomialNB
Media accuracy:  0.8296792859818394
Media precision:  0.8262666707452077
Media recall:  0.9559712346477053
Media f1:  0.8862181399487099

Avaliando FinancialPhraseBank LogisticRegression
Media accuracy:  0.8408584234251689
Media precision:  0.8251852171952881
Media recall:  0.9779923507864684
Media f1:  0.8950150196745774

Avaliando FinancialPhraseBank SVM
Media accuracy:  0.857644566719624
Media precision:  0.8510403410236396
Media recall:  0.963324175824176
Media f1:  0.9036756629227221
